# Loading Data and Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
# Read in the hotel booking data
hotels = pd.read_csv('hotel_bookings.csv')
hotels.shape

(119390, 32)

In [3]:
# create arrival date and day-of-week
month_map = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,
             'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}

# make numeric month (works if arrival_date_month is name or already numeric)
hotels['arrival_month_num'] = hotels['arrival_date_month'].apply(
    lambda m: month_map.get(m, m)
).astype(int)

hotels['arrival_date'] = pd.to_datetime(dict(
    year=hotels['arrival_date_year'].astype(int),
    month=hotels['arrival_month_num'].astype(int),
    day=hotels['arrival_date_day_of_month'].astype(int)
), errors='coerce')

hotels['arrival_day_of_week'] = hotels['arrival_date'].dt.day_name()

# quick check
print(hotels[['arrival_date','arrival_day_of_week']].head())
# ...existing code...

  arrival_date arrival_day_of_week
0   2015-07-01           Wednesday
1   2015-07-01           Wednesday
2   2015-07-01           Wednesday
3   2015-07-01           Wednesday
4   2015-07-01           Wednesday


# Aggregate count of hotel bookings by day

Hotel data set

In [4]:
print(hotels.shape, '\n')
# 119390 rows, each row representing an individual customer booking

print(hotels.columns)
# 32 columns in the dataset

(119390, 35) 

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'arrival_month_num',
       'arrival_date', 'arrival_day_of_week'],
      dtype='object')


Aggregate hotel data set by year, month, week, and day in order to agg
- could potentially use bookings not canceled as demand, and bookings canceled+bookings as potential demand (ask prof)

In [5]:
# Create the new column to represent when room assignemnt is changed by the hotel
hotels['room_assignment_moved'] = np.where(hotels['assigned_room_type'] != hotels['reserved_room_type'], 'moved', 'not moved')

In [6]:
hotels['country'].value_counts().sort_values(ascending=False).head(15)

country
PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
ITA     3766
IRL     3375
BEL     2342
BRA     2224
NLD     2104
USA     2097
CHE     1730
CN      1279
AUT     1263
SWE     1024
Name: count, dtype: int64

In [7]:
# Get dummy variables for categorical variables
hotels_for_agg = pd.get_dummies(hotels, 
                        columns= ['meal', 'market_segment', 'deposit_type', 'customer_type', 'room_assignment_moved', 'hotel', 'assigned_room_type'], 
                        dummy_na=True,
                        drop_first= False)

In [8]:
hotels.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_month_num,arrival_date,arrival_day_of_week,room_assignment_moved
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-15,7,2015-07-01,Wednesday,not moved
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,01-07-15,7,2015-07-01,Wednesday,not moved
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,02-07-15,7,2015-07-01,Wednesday,moved
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,02-07-15,7,2015-07-01,Wednesday,not moved
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,03-07-15,7,2015-07-01,Wednesday,not moved


## Creating daily demand dataset

In [9]:
dbd = hotels_for_agg.groupby(["arrival_date_year","arrival_date_month","arrival_date_week_number","arrival_date_day_of_month", "arrival_date",	"arrival_day_of_week"]).agg(
    # Price and demand columns
    adr = ('adr', 'mean'), # mean average daily rate
    bookings = ('arrival_date_day_of_month', 'count'), # Number of bookings on that day
    canceled = ('is_canceled', 'sum'),

    # Mean Numerically aggregated columns:
    mean_lead_time = ('lead_time', 'mean'),
    mean_week_nights = ('stays_in_week_nights', 'mean'),
    mean_weekend_nights = ('stays_in_weekend_nights', 'mean'),
    mean_adults = ('adults', 'mean'),
    mean_children = ('children', 'mean'),
    mean_babies = ('babies', 'mean'),
    mean_days_in_waiting_list = ('days_in_waiting_list', 'mean'),
    mean_required_car_parking_spaces = ('required_car_parking_spaces', 'mean'),
    mean_total_special_requests = ('total_of_special_requests', 'mean'),

    # Proprtionally aggregated columns:
    # Meal type columns
    bookings_meal_BB = ('meal_BB', 'sum'),
    bookings_meal_FB = ('meal_FB', 'sum'),
    bookings_meal_HB = ('meal_HB', 'sum'),
    bookings_meal_SC = ('meal_SC', 'sum'),
    bookings_meal_undefined = ('meal_Undefined', 'sum'),
    bookings_meal_nan = ('meal_nan', 'sum'),

    # Market type columns
    bookings_market_Aviation = ('market_segment_Aviation', 'sum'),
    bookings_market_Complementary = ('market_segment_Complementary', 'sum'),
    bookings_market_Corporate = ('market_segment_Corporate', 'sum'),
    bookings_market_Direct = ('market_segment_Direct', 'sum'),
    bookings_market_Groups = ('market_segment_Groups', 'sum'),
    bookings_market_Offline_TA_TO = ('market_segment_Offline TA/TO', 'sum'),
    bookings_market_Online_TA = ('market_segment_Online TA', 'sum'),
    bookings_market_Undefined = ('market_segment_Undefined', 'sum'),
    bookings_market_nan = ('market_segment_nan', 'sum'),

    # Deposit type columns
    bookings_deposit_No_Deposit = ('deposit_type_No Deposit', 'sum'),
    bookings_deposit_Non_Refund = ('deposit_type_Non Refund', 'sum'),
    bookings_deposit_Refundable = ('deposit_type_Refundable', 'sum'),
    bookings_deposit_nan = ('deposit_type_nan', 'sum'),

    # Customer type columns
    bookings_customer_Contract = ('customer_type_Contract', 'sum'),
    bookings_customer_Group = ('customer_type_Group', 'sum'),
    bookings_customer_Transient = ('customer_type_Transient', 'sum'),
    bookings_customer_Transient_Party = ('customer_type_Transient-Party', 'sum'),
    bookings_customer_nan = ('customer_type_nan', 'sum'),

    # Room assignment moved columns
    bookings_room_moved = ('room_assignment_moved_moved', 'sum'),
    bookings_room_not_moved = ('room_assignment_moved_not moved', 'sum'),
    bookings_room_nan = ('room_assignment_moved_nan', 'sum'),

    # Hotel type columns
    bookings_hotel_City = ('hotel_City Hotel', 'sum'),
    bookings_hotel_Resort = ('hotel_Resort Hotel', 'sum'),
    bookings_hotel_nan = ('hotel_nan', 'sum'),

    # Assigned room type columns (explicit, same format as other categorical sums)
    bookings_assigned_A = ('assigned_room_type_A', 'sum'),
    bookings_assigned_B = ('assigned_room_type_B', 'sum'),
    bookings_assigned_C = ('assigned_room_type_C', 'sum'),
    bookings_assigned_D = ('assigned_room_type_D', 'sum'),
    bookings_assigned_E = ('assigned_room_type_E', 'sum'),
    bookings_assigned_F = ('assigned_room_type_F', 'sum'),
    bookings_assigned_G = ('assigned_room_type_G', 'sum'),
    bookings_assigned_H = ('assigned_room_type_H', 'sum'),
    bookings_assigned_I = ('assigned_room_type_I', 'sum'),
    bookings_assigned_K = ('assigned_room_type_K', 'sum'),
    bookings_assigned_L = ('assigned_room_type_L', 'sum'),
    bookings_assigned_nan = ('assigned_room_type_nan', 'sum'),

).reset_index()

# Meal proportions
dbd['meal_BB_proportion'] = dbd['bookings_meal_BB'] / dbd['bookings']
dbd['meal_FB_proportion'] = dbd['bookings_meal_FB'] / dbd['bookings']
dbd['meal_HB_proportion'] = dbd['bookings_meal_HB'] / dbd['bookings']
dbd['meal_SC_proportion'] = dbd['bookings_meal_SC'] / dbd['bookings']
dbd['meal_undefined_proportion'] = dbd['bookings_meal_undefined'] / dbd['bookings']
dbd['meal_nan_proportion'] = dbd['bookings_meal_nan'] / dbd['bookings']

# Market segment proportions
dbd['market_Aviation_proportion'] = dbd['bookings_market_Aviation'] / dbd['bookings']
dbd['market_Complementary_proportion'] = dbd['bookings_market_Complementary'] / dbd['bookings']
dbd['market_Corporate_proportion'] = dbd['bookings_market_Corporate'] / dbd['bookings']
dbd['market_Direct_proportion'] = dbd['bookings_market_Direct'] / dbd['bookings']
dbd['market_Groups_proportion'] = dbd['bookings_market_Groups'] / dbd['bookings']
dbd['market_Offline_TA_TO_proportion'] = dbd['bookings_market_Offline_TA_TO'] / dbd['bookings']
dbd['market_Online_TA_proportion'] = dbd['bookings_market_Online_TA'] / dbd['bookings']
dbd['market_Undefined_proportion'] = dbd['bookings_market_Undefined'] / dbd['bookings']
dbd['market_nan_proportion'] = dbd['bookings_market_nan'] / dbd['bookings']

# Deposit type proportions
dbd['deposit_No_Deposit_proportion'] = dbd['bookings_deposit_No_Deposit'] / dbd['bookings']
dbd['deposit_Non_Refund_proportion'] = dbd['bookings_deposit_Non_Refund'] / dbd['bookings']
dbd['deposit_Refundable_proportion'] = dbd['bookings_deposit_Refundable'] / dbd['bookings']
dbd['deposit_nan_proportion'] = dbd['bookings_deposit_nan'] / dbd['bookings']

# Customer type proportions
dbd['customer_Contract_proportion'] = dbd['bookings_customer_Contract'] / dbd['bookings']
dbd['customer_Group_proportion'] = dbd['bookings_customer_Group'] / dbd['bookings']
dbd['customer_Transient_proportion'] = dbd['bookings_customer_Transient'] / dbd['bookings']
dbd['customer_Transient_Party_proportion'] = dbd['bookings_customer_Transient_Party'] / dbd['bookings']
dbd['customer_nan_proportion'] = dbd['bookings_customer_nan'] / dbd['bookings']

# Room assignment proportions
dbd['room_moved_proportion'] = dbd['bookings_room_moved'] / dbd['bookings']
dbd['room_not_moved_proportion'] = dbd['bookings_room_not_moved'] / dbd['bookings']
dbd['room_nan_proportion'] = dbd['bookings_room_nan'] / dbd['bookings']

# Hotel type proportions
dbd['hotel_City_proportion'] = dbd['bookings_hotel_City'] / dbd['bookings']
dbd['hotel_Resort_proportion'] = dbd['bookings_hotel_Resort'] / dbd['bookings']
dbd['hotel_nan_proportion'] = dbd['bookings_hotel_nan'] / dbd['bookings']

# Assigned room type proportions (explicit)
dbd['assigned_A_proportion'] = dbd['bookings_assigned_A'] / dbd['bookings']
dbd['assigned_B_proportion'] = dbd['bookings_assigned_B'] / dbd['bookings']
dbd['assigned_C_proportion'] = dbd['bookings_assigned_C'] / dbd['bookings']
dbd['assigned_D_proportion'] = dbd['bookings_assigned_D'] / dbd['bookings']
dbd['assigned_E_proportion'] = dbd['bookings_assigned_E'] / dbd['bookings']
dbd['assigned_F_proportion'] = dbd['bookings_assigned_F'] / dbd['bookings']
dbd['assigned_G_proportion'] = dbd['bookings_assigned_G'] / dbd['bookings']
dbd['assigned_H_proportion'] = dbd['bookings_assigned_H'] / dbd['bookings']
dbd['assigned_I_proportion'] = dbd['bookings_assigned_I'] / dbd['bookings']
dbd['assigned_K_proportion'] = dbd['bookings_assigned_K'] / dbd['bookings']
dbd['assigned_L_proportion'] = dbd['bookings_assigned_L'] / dbd['bookings']
dbd['assigned_nan_proportion'] = dbd['bookings_assigned_nan'] / dbd['bookings']

# Not canceled bookings
dbd['not_canceled'] = dbd['bookings'] - dbd['canceled']

# Create a revenue columns demand multiplied by price
dbd['revenue'] = dbd['adr'] * dbd['bookings']


Dataset with each row representing one day, contains demand, price, and other potential variables to include in models

In [10]:
# Select columns to be used for modeling and eda
booking_demand = dbd[['arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
        "arrival_date",	"arrival_day_of_week", 'adr',
       'bookings', 'canceled', 'mean_lead_time', 'mean_week_nights',
       'mean_weekend_nights', 'mean_adults', 'mean_children',
       'mean_babies', 'mean_days_in_waiting_list',
       'mean_required_car_parking_spaces', 'mean_total_special_requests','meal_BB_proportion', 'meal_FB_proportion', 'meal_HB_proportion',
       'meal_SC_proportion', 'meal_undefined_proportion',
       'meal_nan_proportion', 'market_Aviation_proportion',
       'market_Complementary_proportion', 'market_Corporate_proportion',
       'market_Direct_proportion', 'market_Groups_proportion',
       'market_Offline_TA_TO_proportion', 'market_Online_TA_proportion',
       'market_Undefined_proportion', 'market_nan_proportion',
       'deposit_No_Deposit_proportion', 'deposit_Non_Refund_proportion',
       'deposit_Refundable_proportion', 'deposit_nan_proportion',
       'customer_Contract_proportion', 'customer_Group_proportion',
       'customer_Transient_proportion', 'customer_Transient_Party_proportion',
       'customer_nan_proportion', 'room_moved_proportion',
       'room_not_moved_proportion', 'room_nan_proportion',
       'hotel_City_proportion', 'hotel_Resort_proportion',
       'hotel_nan_proportion', 'assigned_A_proportion',
       'assigned_B_proportion', 'assigned_C_proportion',
       'assigned_D_proportion', 'assigned_E_proportion',
       'assigned_F_proportion', 'assigned_G_proportion',
       'assigned_H_proportion', 'assigned_I_proportion',
       'assigned_K_proportion', 'assigned_L_proportion',
       'assigned_nan_proportion', 'not_canceled', 'revenue']]


In [11]:
booking_demand
# booking_demand.to_csv('aggregated_hotel_demand.csv')

,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,arrival_date,arrival_day_of_week,adr,bookings,canceled,mean_lead_time,mean_week_nights,mean_weekend_nights,mean_adults,mean_children,mean_babies,mean_days_in_waiting_list,mean_required_car_parking_spaces,mean_total_special_requests,meal_BB_proportion,meal_FB_proportion,meal_HB_proportion,meal_SC_proportion,meal_undefined_proportion,meal_nan_proportion,market_Aviation_proportion,market_Complementary_proportion,market_Corporate_proportion,market_Direct_proportion,market_Groups_proportion,market_Offline_TA_TO_proportion,market_Online_TA_proportion,market_Undefined_proportion,market_nan_proportion,deposit_No_Deposit_proportion,deposit_Non_Refund_proportion,deposit_Refundable_proportion,deposit_nan_proportion,customer_Contract_proportion,customer_Group_proportion,customer_Transient_proportion,customer_Transient_Party_proportion,customer_nan_proportion,room_moved_proportion,room_not_moved_proportion,room_nan_proportion,hotel_City_proportion,hotel_Resort_proportion,hotel_nan_proportion,assigned_A_proportion,assigned_B_proportion,assigned_C_proportion,assigned_D_proportion,assigned_E_proportion,assigned_F_proportion,assigned_G_proportion,assigned_H_proportion,assigned_I_proportion,assigned_K_proportion,assigned_L_proportion,assigned_nan_proportion,not_canceled,revenue
0,2015,August,31,1,2015-08-01,Saturday,141.161000,110,50,95.027273,3.718182,2.200000,1.945455,0.254545,0.000000,0.000000,0.100000,0.754545,0.518182,0.000000,0.481818,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.172727,0.000000,0.363636,0.463636,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.009091,0.000000,0.618182,0.372727,0.0,0.081818,0.918182,0.0,0.318182,0.681818,0.0,0.500000,0.018182,0.045455,0.245455,0.063636,0.036364,0.072727,0.018182,0.000000,0.0,0.0,0.0,60,15527.71
1,2015,August,32,2,2015-08-02,Sunday,138.089600,50,7,97.500000,3.800000,2.240000,2.000000,0.100000,0.080000,0.000000,0.220000,1.360000,0.660000,0.020000,0.320000,0.000000,0.000000,0.0,0.000000,0.060000,0.000000,0.140000,0.000000,0.300000,0.500000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.040000,0.000000,0.840000,0.120000,0.0,0.120000,0.880000,0.0,0.120000,0.880000,0.0,0.440000,0.000000,0.040000,0.320000,0.120000,0.040000,0.040000,0.000000,0.000000,0.0,0.0,0.0,43,6904.48
2,2015,August,32,3,2015-08-03,Monday,89.511913,183,110,150.491803,2.961749,1.202186,1.967213,0.115385,0.032787,0.000000,0.038251,0.442623,0.879781,0.005464,0.114754,0.000000,0.000000,0.0,0.000000,0.016393,0.005464,0.081967,0.590164,0.131148,0.169399,0.005464,0.0,0.765027,0.234973,0.000000,0.0,0.169399,0.010929,0.448087,0.371585,0.0,0.103825,0.896175,0.0,0.661202,0.338798,0.0,0.715847,0.027322,0.043716,0.098361,0.043716,0.043716,0.010929,0.010929,0.005464,0.0,0.0,0.0,73,16380.68
3,2015,August,32,4,2015-08-04,Tuesday,129.595294,51,12,74.470588,4.000000,0.627451,1.901961,0.078431,0.019608,0.000000,0.137255,0.882353,0.745098,0.000000,0.215686,0.039216,0.000000,0.0,0.000000,0.039216,0.039216,0.372549,0.000000,0.176471,0.372549,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.117647,0.019608,0.745098,0.117647,0.0,0.215686,0.784314,0.0,0.235294,0.764706,0.0,0.372549,0.019608,0.098039,0.274510,0.098039,0.098039,0.039216,0.000000,0.000000,0.0,0.0,0.0,39,6609.36
4,2015,August,32,5,2015-08-05,Wednesday,76.946558,154,64,88.285714,2.525974,0.149351,1.935065,0.006579,0.006494,0.000000,0.032468,0.279221,0.948052,0.000000,0.045455,0.006494,0.000000,0.0,0.000000,0.012987,0.233766,0.103896,0.402597,0.051948,0.188312,0.006494,0.0,0.798701,0.201299,0.000000,0.0,0.019481,0.000000,0.292208,0.688312,0.0,0.207792,0.792208,0.0,0.824675,0.175325,0.0,0.714286,0.032468,0.000000,0.194805,0.025974,0.019481,0.000000,0.000000,0.012987,0.0,0.0,0.0,90,11849.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [12]:
hotels
#  hotels.to_csv('raw_hotel_demand.csv')

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_month_num,arrival_date,arrival_day_of_week,room_assignment_moved
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01-07-15,7,2015-07-01,Wednesday,not moved
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,01-07-15,7,2015-07-01,Wednesday,not moved
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,02-07-15,7,2015-07-01,Wednesday,moved
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,02-07-15,7,2015-07-01,Wednesday,not moved
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,03-07-15,7,2015-07-01,Wednesday,not moved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.0,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,06-09-17,8,2017-08-30,Wednesday,not moved
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,07-09-17,8,2017-08-31,Thursday,not moved
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.0,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,07-09-17,8,2017-08-31,Thursday,not moved
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,07-09-17,8,2017-08-31,Thursday,not moved
